In [2]:
# 载入需要的R包
# Load the required packages
library(ggplot2)
library(tidyverse)
library(data.table)
library(ggsci)
library(Biostrings)
library(ape)
library(mgsub)
library(stringi)
library(keras)
library(tensorflow)
library(readxl)
library(plotrix) 
library(ggplotify)
library(patchwork)
library(ggpubr)
library(Hmisc) 
library(caret)

In [3]:
# 对于混合的碱基，用N进行表示
# As for mix-bases, for instance M (Adenine/Cytidine), symbols for mix-bases from genomic sequence were replaced with Ns symbols.
ChangeN <- function(sequence){
    base_seq <- lapply(sequence,function(x){
        x <- toupper(x)
        x <- unlist(str_split(x,""))
        for(i in 1:length(x)){
            if(! x[i] %in% c("A","T","G","C")){
                x[i] <- "N"
            }
        }
        x <- paste0(x,collapse = "")
    })
    base_seq <- unlist(base_seq)
    return(base_seq)
}

In [4]:
# 采用one-hot编码的方式对序列进行编码
# We adopted one-hot encoding method to convert each nucleotide sequence into a binary vector, 
# such as A (1, 0, 0, 0, 0), C (0, 1, 0, 0, 0), G (0, 0, 1, 0, 0), T (0, 0, 0, 1, 0), N (0, 0, 0, 0, 1). 
EncodingSeq <- function(sequence){
    A <- "10000"
    C <- "01000"
    G <- "00100"
    T <- "00010"
    N <- "00001"
    encoded_seq <- lapply(sequence,function(x){
        x <- toupper(x)
        x <- gsub("A",A,x)
        x <- gsub("C",C,x)
        x <- gsub("G",G,x)
        x <- gsub("T",T,x)
        x <- gsub("N",N,x)
    })
    encoded_seq <- unlist(encoded_seq)
    return(encoded_seq)
}

In [5]:
# 提取RNA编辑位点的侧翼序列
# for each RNA editing site, we extracted the flanking sequence of different lengths nucleotides (nt) from a cytidine
Extract_seq <- function(Sequence, Position, Strand, base_len=90){
        base <- Sequence
        len <- str_length(base)
        pos <- Position
        start <- pos-base_len
        end <- pos+base_len
        if(start<1){
            start_str <- paste(rep("N",abs(start)+1),collapse ='')
        }else{
            start_str <- ""
        }
        if(end>len){
            end_str <- paste(rep("N",end-len),collapse ='')
        }else{
            end_str <- ""
        }
        final_base <- paste0(start_str,substr(base,start,end),end_str)
        final_base <- paste0(substr(final_base,1,base_len),substr(final_base,base_len+2,base_len*2+1))
        if(Strand==0){
            final_base <- mgsub(final_base,c("A","T","G","C","a","g","t","c","N","n"), c("T","A","C","G","t","c","a","g","N","n"))
            final_base <- stri_reverse(final_base)
        }
    return(final_base)
}

In [6]:
# 将one-hot编码的序列转换成3维矩阵
# Converting one-hot encoded sequence into 3D matrix
convStringToMatrix <- function(encodedSeqs, seq_len=100){
  # ensure the character type of encodedSeqs
  encodedSeqs <- as.character(encodedSeqs)
  # create the feature matrix:
  x_array <- array(data = 0, dim = c(5,seq_len, length(encodedSeqs)))
  s <- 1 # sequence/instance index
  r <- 1 # row of the matrix, each row represents A,T/U, G, C
  c <- 1 # column of the matrix, each column represents each nucleotide in the 200nt sequence
  j <- 1 # index of character in the one-hot encoded string
  # store each character into the right place of 3D matrix
  while (s <= length(encodedSeqs)) {
    c <- 1
    while (c <= seq_len) {
      r <- 1
      while (r <= 5) {
        x_array[r,c,s] <- as.integer(substr(encodedSeqs[s], j,j))
        r <- r + 1
        j <- j + 1
      }
      c <- c + 1
    }
    s <- s + 1
    j <- 1
  }

  #change the index order of x_array to the one keras package required:
  x_array_new <- aperm(x_array,c(3,2,1))
  return(x_array_new)
}

In [7]:
# 读取一个fasta序列的数据
# load a sequence in fasta format
data <- readDNAStringSet("accd_Chloroplast.fasta")
data <- data.frame(Sequence=data,Strand=1)
data$Sequence <- ChangeN(data$Sequence)

In [8]:
# 提取待预测的C位置信息
# extract the position of cytidines from fasta sequence
editSite <- lapply(1:nrow(data),function(i){
    x <- unlist(str_split(data$Sequence[i],''))
    cDNA.position <- which(x=="C")
    Strand <- data$Strand[i]
    re <- data.frame(Gene=rownames(data)[i],cDNA.position,Strand)
})
editSite <- do.call('rbind',editSite)

In [9]:
# 设置RNA编辑位点的侧翼序列长度，
# setting length of flanking sequence of C-to-U RNA editing sites
flank_len <- 90
# 载入模型
# load a CNN model
model <- load_model_hdf5(paste0("Choose_flank_",flank_len,"_ratio_1.hdf5"))

In [10]:
# 提取C的侧翼序列
# extracted the flanking sequence of 90 lengths nucleotides (nt) from a cytidine 
RNA_input_seq <- c()
for(i in 1:nrow(editSite)){
    Sequence <- data$Sequence[rownames(data)==editSite$Gene[i]]
    Position <- editSite$cDNA.position[i]
    Strand <- editSite$Strand[i]
    Seq <- Extract_seq(Sequence, Position, Strand, base_len=90)
    RNA_input_seq <- c(RNA_input_seq,Seq)
}

In [11]:
encoded_seq  <- EncodingSeq(RNA_input_seq)
encoded_seq2 <- convStringToMatrix(encoded_seq,seq_len=flank_len*2)

In [12]:
# 预测C被编辑的可能性
# calculated probability value of RNA editing site
classes <- model %>% predict(encoded_seq2)

In [13]:
editSite$probability <- classes[,2]

# R Session Information

In [14]:
utils::sessionInfo()

R version 4.3.2 (2023-10-31 ucrt)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 19044)

Matrix products: default


locale:
[1] LC_COLLATE=Chinese (Simplified)_China.utf8 
[2] LC_CTYPE=Chinese (Simplified)_China.utf8   
[3] LC_MONETARY=Chinese (Simplified)_China.utf8
[4] LC_NUMERIC=C                               
[5] LC_TIME=Chinese (Simplified)_China.utf8    

time zone: Asia/Shanghai
tzcode source: internal

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] caret_6.0-94           lattice_0.22-5         Hmisc_5.1-1           
 [4] ggpubr_0.6.0           patchwork_1.1.3        ggplotify_0.1.2       
 [7] plotrix_3.8-4          readxl_1.4.3           tensorflow_2.14.0.9000
[10] keras_2.13.0           stringi_1.8.1          mgsub_1.7.3           
[13] ape_5.7-1              Biostrings_2.70.1      GenomeInfoDb_1.38.1   
[16] XVector_0.42.0         IRanges_2.36.0   